In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score
from pyod.models.lof import LOF
from scipy.io import loadmat

np.random.seed(67)

## Ex 1

### 1.1

In [ ]:
G = nx.Graph()

with open('ca-AstroPh.txt', 'r') as f:
    lines = f.readlines()[:1500]

for line in lines:
    parts = line.strip().split()
    if len(parts) >= 2:
        node1, node2 = parts[0], parts[1]
        if G.has_edge(node1, node2):
            G[node1][node2]['weight'] += 1
        else:
            G.add_edge(node1, node2, weight=1)

print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

### 1.2

In [ ]:
node_features = {}

for node in G.nodes():
    egonet = nx.ego_graph(G, node, radius=1)
    
    N_i = egonet.number_of_nodes() - 1
    E_i = egonet.number_of_edges()
    W_i = sum(d['weight'] for u, v, d in egonet.edges(data=True))
    
    if egonet.number_of_nodes() > 0:
        adj_matrix = nx.to_numpy_array(egonet, weight='weight')
        eigenvalues, _ = np.linalg.eigh(adj_matrix)
        lambda_w_i = np.max(eigenvalues)
    else:
        lambda_w_i = 0
    
    node_features[node] = {
        'N_i': N_i,
        'E_i': E_i,
        'W_i': W_i,
        'lambda_w_i': lambda_w_i
    }

nx.set_node_attributes(G, node_features)

print(f"Features extracted for {len(node_features)} nodes")

### 1.3

In [ ]:
nodes = list(G.nodes())
N_values = np.array([G.nodes[n]['N_i'] for n in nodes])
E_values = np.array([G.nodes[n]['E_i'] for n in nodes])

valid_mask = (N_values > 0) & (E_values > 0)
valid_nodes = [nodes[i] for i in range(len(nodes)) if valid_mask[i]]
N_valid = N_values[valid_mask]
E_valid = E_values[valid_mask]

log_N = np.log(N_valid).reshape(-1, 1)
log_E = np.log(E_valid)

reg = LinearRegression()
reg.fit(log_N, log_E)

theta = reg.coef_[0]
log_C = reg.intercept_
C = np.exp(log_C)

print(f"Power-law parameters: C = {C:.4f}, theta = {theta:.4f}")

E_pred = C * (N_valid ** theta)

anomaly_scores = {}
for i, node in enumerate(valid_nodes):
    y_i = E_valid[i]
    y_pred = E_pred[i]
    
    score = (max(y_i, y_pred) / (min(y_i, y_pred) + 1e-8)) * np.log(abs(y_i - y_pred) + 1)
    anomaly_scores[node] = score

for node in nodes:
    if node not in anomaly_scores:
        anomaly_scores[node] = 0

### 1.4

In [ ]:
sorted_nodes = sorted(anomaly_scores.items(), key=lambda x: x[1], reverse=True)
top_10_anomalies = set([n for n, s in sorted_nodes[:10]])

node_colors = ['red' if n in top_10_anomalies else 'lightblue' for n in G.nodes()]

plt.figure(figsize=(12, 10))
nx.draw(G, node_color=node_colors, node_size=50, with_labels=False)
plt.title('Graph with Top 10 Anomalies (OddBall - E vs N)')
plt.show()

print("Top 10 anomalous nodes:")
for node, score in sorted_nodes[:10]:
    print(f"  Node {node}: score = {score:.4f}")

### 1.5

In [ ]:
features_for_lof = np.column_stack([N_valid, E_valid])

lof = LOF()
lof.fit(features_for_lof)
lof_scores = lof.decision_scores_

oddball_scores = np.array([anomaly_scores[n] for n in valid_nodes])
oddball_normalized = (oddball_scores - oddball_scores.min()) / (oddball_scores.max() - oddball_scores.min() + 1e-8)
lof_normalized = (lof_scores - lof_scores.min()) / (lof_scores.max() - lof_scores.min() + 1e-8)

combined_scores = oddball_normalized + lof_normalized

combined_anomaly_scores = {}
for i, node in enumerate(valid_nodes):
    combined_anomaly_scores[node] = combined_scores[i]

for node in nodes:
    if node not in combined_anomaly_scores:
        combined_anomaly_scores[node] = 0

sorted_combined = sorted(combined_anomaly_scores.items(), key=lambda x: x[1], reverse=True)
top_10_combined = set([n for n, s in sorted_combined[:10]])

node_colors_combined = ['red' if n in top_10_combined else 'lightblue' for n in G.nodes()]

plt.figure(figsize=(12, 10))
nx.draw(G, node_color=node_colors_combined, node_size=50, with_labels=False)
plt.title('Graph with Top 10 Anomalies (OddBall + LOF)')
plt.show()

print("Top 10 anomalous nodes (combined score):")
for node, score in sorted_combined[:10]:
    print(f"  Node {node}: score = {score:.4f}")

## Ex 2

### 2.1

In [ ]:
G_regular = nx.random_regular_graph(3, 100, seed=42)

G_caveman = nx.connected_caveman_graph(10, 20)

G_merged = nx.union(G_regular, G_caveman, rename=('R', 'C'))

regular_nodes = [n for n in G_merged.nodes() if str(n).startswith('R')]
caveman_nodes = [n for n in G_merged.nodes() if str(n).startswith('C')]

for _ in range(10):
    n1 = np.random.choice(regular_nodes)
    n2 = np.random.choice(caveman_nodes)
    G_merged.add_edge(n1, n2)

print(f"Merged graph - Nodes: {G_merged.number_of_nodes()}, Edges: {G_merged.number_of_edges()}")

plt.figure(figsize=(12, 10))
nx.draw(G_merged, node_size=30, with_labels=False)
plt.title('Merged Graph (Regular + Caveman)')
plt.show()

In [ ]:
node_features_merged = {}

for node in G_merged.nodes():
    egonet = nx.ego_graph(G_merged, node, radius=1)
    N_i = egonet.number_of_nodes() - 1
    E_i = egonet.number_of_edges()
    node_features_merged[node] = {'N_i': N_i, 'E_i': E_i}

nodes_m = list(G_merged.nodes())
N_m = np.array([node_features_merged[n]['N_i'] for n in nodes_m])
E_m = np.array([node_features_merged[n]['E_i'] for n in nodes_m])

valid_mask_m = (N_m > 0) & (E_m > 0)
valid_nodes_m = [nodes_m[i] for i in range(len(nodes_m)) if valid_mask_m[i]]
N_valid_m = N_m[valid_mask_m]
E_valid_m = E_m[valid_mask_m]

log_N_m = np.log(N_valid_m).reshape(-1, 1)
log_E_m = np.log(E_valid_m)

reg_m = LinearRegression()
reg_m.fit(log_N_m, log_E_m)
theta_m = reg_m.coef_[0]
C_m = np.exp(reg_m.intercept_)

E_pred_m = C_m * (N_valid_m ** theta_m)

scores_m = {}
for i, node in enumerate(valid_nodes_m):
    y_i = E_valid_m[i]
    y_pred = E_pred_m[i]
    score = (max(y_i, y_pred) / (min(y_i, y_pred) + 1e-8)) * np.log(abs(y_i - y_pred) + 1)
    scores_m[node] = score

for node in nodes_m:
    if node not in scores_m:
        scores_m[node] = 0

sorted_m = sorted(scores_m.items(), key=lambda x: x[1], reverse=True)
top_10_clique = set([n for n, s in sorted_m[:10]])

colors_m = ['red' if n in top_10_clique else 'lightblue' for n in G_merged.nodes()]

plt.figure(figsize=(12, 10))
nx.draw(G_merged, node_color=colors_m, node_size=30, with_labels=False)
plt.title('Clique Detection - Top 10 Anomalies')
plt.show()

print("Top 10 nodes (likely part of cliques):")
for node, score in sorted_m[:10]:
    print(f"  Node {node}: score = {score:.4f}")

### 2.2

In [ ]:
G1 = nx.random_regular_graph(3, 100, seed=42)
G2 = nx.random_regular_graph(5, 100, seed=43)

G_heavy = nx.union(G1, G2, rename=('A', 'B'))

for edge in list(G_heavy.edges()):
    G_heavy.add_edge(edge[0], edge[1], weight=1)

A_nodes = [n for n in G_heavy.nodes() if str(n).startswith('A')]
B_nodes = [n for n in G_heavy.nodes() if str(n).startswith('B')]

for _ in range(5):
    n1 = np.random.choice(A_nodes)
    n2 = np.random.choice(B_nodes)
    G_heavy.add_edge(n1, n2, weight=1)

random_nodes = np.random.choice(list(G_heavy.nodes()), 2, replace=False)
heavy_nodes = set(random_nodes)

for node in random_nodes:
    egonet = nx.ego_graph(G_heavy, node, radius=1)
    for u, v in egonet.edges():
        G_heavy[u][v]['weight'] += 10

print(f"Heavy vicinity nodes: {random_nodes}")

In [ ]:
node_features_heavy = {}

for node in G_heavy.nodes():
    egonet = nx.ego_graph(G_heavy, node, radius=1)
    E_i = egonet.number_of_edges()
    W_i = sum(d['weight'] for u, v, d in egonet.edges(data=True))
    node_features_heavy[node] = {'E_i': E_i, 'W_i': W_i}

nodes_h = list(G_heavy.nodes())
E_h = np.array([node_features_heavy[n]['E_i'] for n in nodes_h])
W_h = np.array([node_features_heavy[n]['W_i'] for n in nodes_h])

valid_mask_h = (E_h > 0) & (W_h > 0)
valid_nodes_h = [nodes_h[i] for i in range(len(nodes_h)) if valid_mask_h[i]]
E_valid_h = E_h[valid_mask_h]
W_valid_h = W_h[valid_mask_h]

log_E_h = np.log(E_valid_h).reshape(-1, 1)
log_W_h = np.log(W_valid_h)

reg_h = LinearRegression()
reg_h.fit(log_E_h, log_W_h)
theta_h = reg_h.coef_[0]
C_h = np.exp(reg_h.intercept_)

W_pred_h = C_h * (E_valid_h ** theta_h)

scores_h = {}
for i, node in enumerate(valid_nodes_h):
    y_i = W_valid_h[i]
    y_pred = W_pred_h[i]
    score = (max(y_i, y_pred) / (min(y_i, y_pred) + 1e-8)) * np.log(abs(y_i - y_pred) + 1)
    scores_h[node] = score

for node in nodes_h:
    if node not in scores_h:
        scores_h[node] = 0

sorted_h = sorted(scores_h.items(), key=lambda x: x[1], reverse=True)
top_4_heavy = set([n for n, s in sorted_h[:4]])

colors_h = ['red' if n in top_4_heavy else 'lightblue' for n in G_heavy.nodes()]

plt.figure(figsize=(12, 10))
nx.draw(G_heavy, node_color=colors_h, node_size=30, with_labels=False)
plt.title('Heavy Vicinity Detection - Top 4 Anomalies')
plt.show()

print("Top 4 nodes (heavy vicinity):")
for node, score in sorted_h[:4]:
    print(f"  Node {node}: score = {score:.4f}")

## Ex 3

### 3.1-3.2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import from_scipy_sparse_matrix

In [ ]:
mat_data = loadmat('ACM.mat')

X = mat_data['Attributes']
if hasattr(X, 'toarray'):
    X = X.toarray()
X = torch.tensor(X, dtype=torch.float32)

A_sparse = mat_data['Network']
edge_index, edge_weight = from_scipy_sparse_matrix(A_sparse)

A = torch.tensor(A_sparse.toarray(), dtype=torch.float32)

y = mat_data['Label'].ravel()
y = torch.tensor(y, dtype=torch.float32)

print(f"Node attributes shape: {X.shape}")
print(f"Edge index shape: {edge_index.shape}")
print(f"Adjacency matrix shape: {A.shape}")
print(f"Labels shape: {y.shape}")
print(f"Number of anomalies: {int(y.sum())} ({y.sum()/len(y)*100:.2f}%)")

### 3.3

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim):
        super(Encoder, self).__init__()
        self.conv1 = GCNConv(input_dim, 128)
        self.conv2 = GCNConv(128, 64)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        return x


class AttributeDecoder(nn.Module):
    def __init__(self, output_dim):
        super(AttributeDecoder, self).__init__()
        self.conv1 = GCNConv(64, 128)
        self.conv2 = GCNConv(128, output_dim)
    
    def forward(self, z, edge_index):
        z = self.conv1(z, edge_index)
        z = F.relu(z)
        z = self.conv2(z, edge_index)
        z = F.relu(z)
        return z


class StructureDecoder(nn.Module):
    def __init__(self):
        super(StructureDecoder, self).__init__()
        self.conv = GCNConv(64, 64)
    
    def forward(self, z, edge_index):
        z = self.conv(z, edge_index)
        z = F.relu(z)
        A_hat = z @ z.T
        return A_hat


class GraphAutoencoder(nn.Module):
    def __init__(self, input_dim):
        super(GraphAutoencoder, self).__init__()
        self.encoder = Encoder(input_dim)
        self.attr_decoder = AttributeDecoder(input_dim)
        self.struct_decoder = StructureDecoder()
    
    def forward(self, x, edge_index):
        z = self.encoder(x, edge_index)
        x_hat = self.attr_decoder(z, edge_index)
        a_hat = self.struct_decoder(z, edge_index)
        return x_hat, a_hat, z

### 3.4

In [ ]:
def gae_loss(X, X_hat, A, A_hat, alpha=0.8):
    attr_loss = torch.norm(X - X_hat, p='fro') ** 2
    struct_loss = torch.norm(A - A_hat, p='fro') ** 2
    return alpha * attr_loss + (1 - alpha) * struct_loss

### 3.5

In [ ]:
input_dim = X.shape[1]
model = GraphAutoencoder(input_dim)

optimizer = torch.optim.Adam(model.parameters(), lr=0.004)

alpha = 0.8
n_epochs = 100

losses = []
aucs = []

for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    
    X_hat, A_hat, z = model(X, edge_index)
    
    loss = gae_loss(X, X_hat, A, A_hat, alpha)
    
    loss.backward()
    
    optimizer.step()
    
    losses.append(loss.item())
    
    if (epoch + 1) % 5 == 0:
        model.eval()
        with torch.no_grad():
            X_hat, A_hat, z = model(X, edge_index)
            
            attr_error = torch.sum((X - X_hat) ** 2, dim=1)
            struct_error = torch.sum((A - A_hat) ** 2, dim=1)
            scores = alpha * attr_error + (1 - alpha) * struct_error
            
            auc = roc_auc_score(y.numpy(), scores.numpy())
            aucs.append(auc)
            print(f"Epoch {epoch+1}/{n_epochs} - Loss: {loss.item():.4f} - ROC AUC: {auc:.4f}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(losses)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True, alpha=0.3)

axes[1].plot(range(5, n_epochs+1, 5), aucs)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('ROC AUC')
axes[1].set_title('ROC AUC Score')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final ROC AUC: {aucs[-1]:.4f}")